In [5]:
import pandas as pd
import numpy as np


!export JAVA_HOME=/usr/lib/jvm/java-18-openjdk-amd64/


In [4]:
df = pd.read_csv('./data/final_evaluation_set.csv')
df.nunique(axis=0)


Unnamed: 0    971
Language        1
Query          99
GitHubUrl     958
code          958
relevance       4
dtype: int64

In [6]:
# Create index
from collections import defaultdict
import pyterrier as pt
if not pt.started():
    pt.init()
    

visited_code = []    
count = 0
code_dic = {}
temp_dic = dict({'text':[], 'docno':[]})
for i in range(len(df['code'])):
    if df['code'][i] not in visited_code:
        visited_code.append(df['code'][i])
        code_dic[df['code'][i]] = str(count)
        temp_dic['text'].append(df['code'][i])
        temp_dic['docno'].append(str(count))
        count += 1
    
code_df = pd.DataFrame(temp_dic)    


print(code_df.head(5))
code_df.drop_duplicates(subset='text')
pd_indexer = pt.DFIndexer('./pd_index', overwrite=True)
indexref = pd_indexer.index(code_df["text"], code_df["docno"])
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



                                                text docno
0  def write_csv(filename, data, delimiter=CSV_DE...     0
1  def write_to_csv(fname, header, rows): with op...     1
2  def write_csv(self): # Get first element's fil...     2
3  def to_csv(self, fbuf, quotechar='"', delimite...     3
4  def write_to_csv(self, filename): ''' Exports ...     4
Number of documents: 958
Number of terms: 4929
Number of postings: 26358
Number of fields: 0
Number of tokens: 65017
Field names: []
Positions:   false



No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [7]:
# Prepare query data
visited = []    
count = 0
query_dic = {}
temp_query_dic = dict({'qid':[], 'query':[],})
for i in range(len(df['Query'])):
    if df['Query'][i] not in visited:
        visited.append(df['Query'][i])
        query_dic[df['Query'][i]] = str(count)
        temp_query_dic['query'].append(df['Query'][i])
        temp_query_dic['qid'].append(str(count))
        count += 1
    
query_df = pd.DataFrame(temp_query_dic)    

print(query_df.head(5))
print(len(query_df))

query_df.query("query == 'write csv'")
type(query_df.iloc[0,1])

  qid                           query
0   0                       write csv
1   1           unzipping large files
2   2                 unique elements
3   3  underline text in label widget
4   4                  string to date
99


str

In [8]:
print(len(df))
pair_dic = dict({'qid':[], 'docno':[], 'label':[]})
for i in range(len(df)):
    qid = int(query_df[query_df['query']==df['Query'][i]]['qid'])
    pair_dic['qid'].append(str(qid))
    docno = int(code_df[code_df['text']==df['code'][i]]['docno'])
    pair_dic['docno'].append(str(docno))
    pair_dic['label'].append(df['relevance'][i])

qrels = pd.DataFrame(pair_dic)
print(qrels.head(5))

971
  qid docno  label
0   0     0      3
1   0     1      3
2   0     2      3
3   0     3      3
4   0     4      3


In [9]:
# Prepare eval data
print(len(df))
pair_dic = dict({'qid':[], 'docno':[], 'label':[]})
for i in range(len(df)):
    qid = int(query_df[query_df['query']==df['Query'][i]]['qid'])
    pair_dic['qid'].append(str(qid))
    docno = int(code_df[code_df['text']==df['code'][i]]['docno'])
    pair_dic['docno'].append(str(docno))
    pair_dic['label'].append(df['relevance'][i])

qrels = pd.DataFrame(pair_dic)

971


In [ ]:
# learning to Rank
SEED=42
from sklearn.model_selection import train_test_split
tr_va_topics, test_topics = train_test_split(query_df, test_size=30, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=10, random_state=SEED)
print(test_topics.head(5))
# print(test_topics.dtypes)
# pd.to_numeric(test_topics['qid'], downcast='integer')
# print(test_topics.dtypes)
test_topics.iloc[1,0]
print(test_topics.shape)

# for i in range(len(test_topics)):
#     tmp = int(test_topics[i][0])
#     print(tmp, type(tmp))
#     test_topics[i][0] = tmp
# print(test_topics.dtypes)